In [6]:
import scrape
import os

In [7]:
discover_dir = '/home/bhanu/Documents/projects/personal/learning/discover-container/discover'
discover_nlp_path = os.path.join(discover_dir, 'scrape.py')
discover_mod = scrape.load_module(discover_nlp_path)
[c for c in scrape.find_functions(discover_mod)]

[<function module.name._func_calls(fn)>,
 <function module.name._method_calls(fn)>,
 <function module.name.annotate(func)>,
 <function module.name.camel_case_split(identifier)>,
 <function module.name.find_functions(module)>,
 <function module.name.find_modules(x)>,
 <function module.name.index_python_files(py_module)>,
 <function module.name.load_module(path)>,
 <function module.name.process_docstring(doc_string)>,
 <function module.name.process_funcs(functions)>,
 <function module.name.process_methods(methods)>,
 <function module.name.process_name(func_name)>,
 <function module.name.snake_case_split(identifier)>,
 <function module.name.tokenize(annotation)>]

In [4]:
def tokenize(annotation):
    tokens = []
    tokens.extend()

In [8]:
for fn in scrape.find_functions(discover_mod):
    print("fn", fn.__name__)
    for k, v in scrape.annotate(fn).items():
        print("\t", k)
        print("\t", "*" * 30)
        if v:
            print("\t", v)
        print("\n")

fn _func_calls
	 doc
	 ******************************
	 Determine function calls within a function

    Args:
        fn (module): python module

    Returns:
        funcs (list): list of function calls within a function
    


	 name
	 ******************************
	 _func_calls


	 functions
	 ******************************
	 ['dis', 'Bytecode', 'append']


	 methods
	 ******************************
	 ['Bytecode', 'opname', 'append', 'argval']


fn _method_calls
	 doc
	 ******************************
	 Determine method calls within a function

    Args:
        fn (module): python module

    Returns:
        (list): list of method calls within a function
    


	 name
	 ******************************
	 _method_calls


	 functions
	 ******************************
	 ['re', 'findall', 'METHOD', 'getsource']


	 methods
	 ******************************
	 ['findall']


fn annotate
	 doc
	 ******************************
	 Annotate a function with doc string, functions, methods and name


In [15]:
for fn in scrape.find_functions(discover_mod):
    print(fn)
    for mc in scrape._method_calls(fn):
        print("\t", mc)
    print("*" * 20)

<function _func_calls at 0x7f8fdceedb00>
	 Bytecode
	 opname
	 append
	 argval
********************
<function _method_calls at 0x7f8fdceeda70>
	 findall
********************
<function annotate at 0x7f8fdceedb90>
	 __doc__
	 __doc__
	 __name__
	 __class__
	 __class__
********************
<function camel_case_split at 0x7f901c1214d0>
	 finditer
	 group
********************
<function find_functions at 0x7f8fdceed9e0>
	 __name__
********************
<function find_modules at 0x7f8fdceed950>
	 rglob
	 py
********************
<function index_python_files at 0x7f8fdceed830>
	 append
	 join
	 fit_transform
	 npz
	 pkl
********************
<function load_module at 0x7f8fdceed8c0>
	 name
	 loader
	 exec_module
	 loader
	 get_code
	 name
********************
<function process_docstring at 0x7f8fdceede60>
	 findall
	 extend
********************
<function process_funcs at 0x7f8fdceedd40>
	 extend
	 extend
********************
<function process_methods at 0x7f8fdceedcb0>
	 extend
*******************

In [18]:
documents = []

for fn in scrape.find_functions(discover_mod):
    ann = scrape.annotate(fn)
    documents.append(' '.join(scrape.tokenize(ann)))

documents

['determine function calls within a function args fn module python module returns funcs list list of function calls within a function  func calls dis bytecode append bytecode opname append argval',
 'determine method calls within a function args fn module python module returns list list of method calls within a function  method calls re findall method getsource findall',
 'annotate a function with doc string functions methods and name args func module python module returns d dict description annotate type  func calls type  method calls   doc     doc     name     class     class  ',
 'split camel case function names to tokens args identifier str identifier to split returns list lower case split tokens ex camel case camel case split re finditer finditer group',
 'determine method calls within a function args module module python module returns attr list list of method calls within a function find functions dir getattr callable getattr getattr   name  ',
 'get all python files given a pat